1\. Write a function that converts number representation, bin<->dec<->hex. (Clearly using the corresponding python built-in functions is not fair..)

In [1]:
def DecToBin(n):
    remainders=[]
    while int(n)>0:
        remainders.append(str(n%2))
        n=n//2
    return "".join(remainders[::-1])
    
def BinToDec(s):
    d=0
    for i,c in enumerate(s[::-1]):
        d+=int(c)*2**i
    return d

def DecToHex(n):
    remainders=[]
    while int(n)>0:
        remainder=n%16
        if remainder<10:
            remainders.append(str(remainder))
        else:
            remainders.append(str(chr(55+remainder)))
        n=n//16
    return "".join(remainders[::-1])
    
def HexToDec(s):
    d=0
    for i,c in enumerate(s[::-1]):
        if c.isdigit():
            d+=int(c)*16**i
        else:
            d+=int(ord(c)-55)*16**i
    return d

def HexToBin(s):
    return DecToBin(HexToDec(s))

def BinToHex(s):
    return DecToHex(BinToDec(s))

DecToBin(42), BinToDec(DecToBin(42)), BinToDec('000101'), DecToHex(HexToDec('FADC0E'))

('101010', 42, 5, 'FADC0E')

2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

Take a look [here](https://www.h-schmidt.net/FloatConverter/IEEE754.html) to check

In [2]:
def WordToFloat(s):
    '''
    This function takes a word as input and returns the float values it 
    stands for.
    
    A word is defined as an integer value returned by a sensor.
    Such word/integer value has to be parsed according to the IEEE-754
    standard.
    
    MSB
    1-bit:   sign
    8-bits:  exponent, with bias of -127
    23-bits: mantissa
    LSB
    
    Special care should be taken when the exponent bits are all set
    to 0, as a different convention is used to express the number '0',
    namely a bias of -126 is used and the addition of the hidden '1' 
    bit to the mantissa is not performed.
    '''
    sgn_mask=2147483648
    exp_mask=2139095040
    man_mask=8388607
    
    sgn=(s&sgn_mask)>>31
    exp=(s&exp_mask)>>23
    man=(s&man_mask)
    
    d_man=man/(2**23)
    print('sgn',sgn)
    print('exp',exp)
    print('man', man)
    print('d_man', d_man)
    if exp!=0:
        return ((-1)**sgn)*(2**(exp-127))*(1+d_man)
    else:
        return ((-1)**sgn)*(2**(exp-126))*(d_man)

word=int('01000010110010000000000000000000',2)
print('Word:',word, '\nNumber:', WordToFloat(word))

sgn 0
exp 133
man 4718592
d_man 0.5625
Word: 1120403456 
Number: 100.0


In [3]:
#examples of masking
sgn_mask=int('10000000000000000000000000000000',2)
exp_mask=int('01111111100000000000000000000000',2)
man_mask=int('00000000011111111111111111111111',2)
test_123=int('11011001111001111111111111111111',2)

ss=(test_123&sgn_mask)>>31
ee=(test_123&exp_mask)>>23
mm=test_123&man_mask

for num in [sgn_mask, exp_mask, man_mask, test_123, ss, ee, mm]:
    print('{let:>15}\t{binary:032b}'.format(let=num, binary=(num)))

     2147483648	10000000000000000000000000000000
     2139095040	01111111100000000000000000000000
        8388607	00000000011111111111111111111111
     3655860223	11011001111001111111111111111111
              1	00000000000000000000000000000001
            179	00000000000000000000000010110011
        6815743	00000000011001111111111111111111


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [4]:
#solution 1
underflow=1.0
i=0

while True:
    i+=1
    if underflow/2==0:
        print(i, underflow, underflow/2)
        break
    else:
        underflow=underflow/2

#solution 2
u=1
while u>0:
    if u/2==0:
        print(u)
    u/=2

1075 5e-324 0.0
5e-324


In [5]:
#solution 1
import math
overflow=1.0
i=0

while True:
    i+=1
    if overflow*2==math.inf:
        print(i, overflow, overflow*2)
        break
    else:
        overflow*=2
        
#solution 2       
u=1.0
while u<math.inf:
    if u*2==math.inf:
        print(u)
    u*=2

1024 8.98846567431158e+307 inf
8.98846567431158e+307


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [6]:
a=1.0
i=1
while True:
    if a==a+2**(-i):
        print('So... the limit is 2^-'+str(i) )
        break
    i+=1

So... the limit is 2^-53


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [7]:
import numpy as np

def precision(func):
    def wrapper(*args, **kwargs):
        x1,x2=func(*args, **kwargs)
        if kwargs['b']>0:
            print(func.__name__, '{a:.6e} {b:.6e}'.format(a=abs(x1+1e-6)/(1e-6), b=abs(x2+1e+6)/(1e+6)))
        else:
            print(func.__name__, '{a:.6e} {b:.6e}'.format(a=abs(x1-1e+6)/(1e+6), b=abs(x2-1e-6)/(1e-6)))
        return x1,x2
    return wrapper

@precision
def roots1(a,b,c):
    delta=b*b-4.*a*c
    x1=(-b+delta**(0.5))/(2.*a)
    x2=(-b-delta**(0.5))/(2.*a)
    return x1,x2

@precision
def roots2(a,b,c):
    num=4.*a*c
    delta=b*b-4*a*c
    x1=num/(-2.*a*b-((2*a*b)**2-16*a**3*c)**(0.5))
    x2=num/(-2.*a*b+((2*a*b)**2-16*a**3*c)**(0.5))
    return x1,x2

@precision
def roots3(a,b,c):
    num=4*a*c
    delta=b*b-4*a*c
    x1=4*a*c/(-2*a*b-2*a*(delta)**(0.5))
    x2=4*a*c/(-2*a*b+2*a*(delta)**(0.5))
    return x1,x2

@precision
def roots4(a,b,c):
    num=4*a*c
    delta=b*b-4*a*c
    x1=4*a*c/(-2*a*b-2*a*(delta)**(0.5))
    x2=(-b-delta**(0.5))/(2.*a)
    return x1,x2

@precision
def roots5(a,b,c):
    num=4*a*c
    delta=b*b-4*a*c
    if b>0:
        x1=4*a*c/(-2*a*b-2*a*(delta)**(0.5))
        x2=(-b-delta**(0.5))/(2.*a)
    else:
        x1=(-b+delta**(0.5))/(2.*a)
        x2=4*a*c/(-2*a*b+2*a*(delta)**(0.5))

    return x1,x2

a=0.001
b=1000
c=0.001
(roots1(a=a,b=b,c=c), roots2(a=a,b=b,c=c), roots3(a=a,b=b,c=c), roots4(a=a,b=b,c=c), roots5(a=a,b=b,c=c))

roots1 1.057540e-05 1.000008e-12
roots2 1.000134e-12 3.338832e-05
roots3 9.999224e-13 2.212221e-05
roots4 9.999224e-13 1.000008e-12
roots5 9.999224e-13 1.000008e-12


((-9.999894245993346e-07, -999999.999999),
 (-1.000000000001e-06, -999966.6116837071),
 (-1.0000000000009999e-06, -1000022.1222095027),
 (-1.0000000000009999e-06, -999999.999999),
 (-1.0000000000009999e-06, -999999.999999))

# Answer to the question
Apparently a greater precision is achieved in the computation of the roots of the polynomial if both the square root of the determinant $\Delta$ and the $b$ coefficient are of the same sign e.g. both positive or negative.


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [8]:
def f(x):
    return x*(x-1)

def der_def(delta, f, x):
    return (f(x+delta)-f(x))/delta

deltas=[10**(-2*i) for i in range(1,8)]
for delta in deltas:
    d=der_def(delta,f, 1)
    print('Delta: {delta:.2e}\t Der: {der:.10f}\tErr: {err:.2e}'.format(delta=delta, der=d, err=(abs(d-1)/1*100)) )

Delta: 1.00e-02	 Der: 1.0100000000	Err: 1.00e+00
Delta: 1.00e-04	 Der: 1.0001000000	Err: 1.00e-02
Delta: 1.00e-06	 Der: 1.0000009999	Err: 1.00e-04
Delta: 1.00e-08	 Der: 1.0000000039	Err: 3.92e-07
Delta: 1.00e-10	 Der: 1.0000000828	Err: 8.28e-06
Delta: 1.00e-12	 Der: 1.0000889006	Err: 8.89e-03
Delta: 1.00e-14	 Der: 0.9992007222	Err: 7.99e-02


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [9]:
import math
import time
def func(x):
    return (1-x**2)**(0.5)

def timer(func):
    def func_wrapper(*args, **kwargs):
        t_s=time.time()
        ret=func(*args, **kwargs)
        t_total=time.time()-t_s
        print('Total time: %.2e sec'%t_total)
        return ret
    return func_wrapper

@timer
def integrate(start, stop, N, func):
    dx=(stop-start)/N
    integral=0
    for i in range(N):
        x=start+dx*i
        integral+=func(x)*dx
    return integral

def perc_error(i,N):
    pi_half=integrate(-1,1,N, func)
    perc=abs(pi_half- math.pi/2)/(math.pi/2)*100
    print('Res: ',pi_half, '\tErr: %.1e'%perc,'%','\nTru: ', math.pi/2, '\tSub: 10^',i, sep='')

subdiv=[10**i for i in range(2,9)]
subdiv.append(2*100000000)
for i,n in enumerate(subdiv):
    perc_error(i+2,n)

Total time: 1.73e-04 sec
Res: 1.5691342555492505	Err: 1.1e-01%
Tru: 1.5707963267948966	Sub: 10^2
Total time: 1.19e-03 sec
Res: 1.570743738501071	Err: 3.3e-03%
Tru: 1.5707963267948966	Sub: 10^3
Total time: 8.11e-03 sec
Res: 1.570794663715291	Err: 1.1e-04%
Tru: 1.5707963267948966	Sub: 10^4
Total time: 4.66e-02 sec
Res: 1.5707962742034223	Err: 3.3e-06%
Tru: 1.5707963267948966	Sub: 10^5
Total time: 3.64e-01 sec
Res: 1.5707963251317274	Err: 1.1e-07%
Tru: 1.5707963267948966	Sub: 10^6
Total time: 3.25e+00 sec
Res: 1.5707963267423612	Err: 3.3e-09%
Tru: 1.5707963267948966	Sub: 10^7
Total time: 3.14e+01 sec
Res: 1.570796326793284	Err: 1.0e-10%
Tru: 1.5707963267948966	Sub: 10^8
Total time: 5.74e+01 sec
Res: 1.5707963267944416	Err: 2.9e-11%
Tru: 1.5707963267948966	Sub: 10^9
